In [ ]:
from azureml.core import Run,Workspace
from azureml.core.experiment import Experiment
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import json


In [ ]:
# Load workspace
ws = Workspace.from_config()


In [ ]:
classification_tests  = []
# NoWindow
test_no_window = {}
test_no_window['ExperimentName'] = 'NoWindow'
test_no_window['Comment'] = 'Imbalanced dataset, PNG image generated without futher window and level.'
test_no_window['SoreRunId'] = 'NoWindow_1622062464_86b235aa'
test_no_window['TestDataset'] = 'NoWindow_test'
test_no_window['TestDatasetId'] = '2a63d9d0-dd6b-479d-bc01-da7a7b065ac4'
test_no_window['y_pred'] = []
test_no_window['y_true'] = []
classification_tests.append(test_no_window)

# balanced
test_balanced = {}
test_balanced['ExperimentName'] = 'Balanced_NoWindow'
test_balanced['Comment'] = 'Balanced dataset, PNG image generated without futher window and level.'
test_balanced['SoreRunId'] = 'Balanced_NoWindow_1622103656_29450fdf'
test_balanced['TestDataset'] = 'Balanced_NoWindow_test'
test_balanced['TestDatasetId'] = '2d8d0910-90ce-4c02-81d4-55abb2f8778c'
test_balanced['y_pred'] = []
test_balanced['y_true'] = []
classification_tests.append(test_balanced)

# 100by100
test_100by100 = {}
test_100by100['ExperimentName'] = '100by100'
test_100by100['Comment'] = 'Imbalanced dataset, PNG image generated with window center=100, window width = 100.'
test_100by100['SoreRunId'] = '100by100_1622086790_52811ae4'
test_100by100['TestDataset'] = '100by100_test'
test_100by100['TestDatasetId'] = 'bc69d1e3-2f54-420d-856d-69de440e3815'
test_100by100['y_pred'] = []
test_100by100['y_true'] = []
classification_tests.append(test_100by100)

# watermark
test_watermark = {}
test_watermark['ExperimentName'] = 'WaterMarked'
test_watermark['Comment'] = 'Balanced dataset, PNG image generated without futher window and level, with WaterMark of Gender and ViewPosition.'
test_watermark['SoreRunId'] = 'WaterMarked_1622526402_155992c8'
test_watermark['TestDataset'] = 'WaterMarked_test'
test_watermark['TestDatasetId'] = '44289778-b8e6-40c0-9334-f950e6c874e9'
test_watermark['y_pred'] = []
test_watermark['y_true'] = []
classification_tests.append(test_watermark)

In [ ]:
#
def get_predictions(experiment_name: str, run_id: str) -> dict:
    experiment = Experiment(ws, experiment_name)
    scoring_run = Run(experiment, run_id)
    output_prediction_file = "./outputs/predictions.txt"

    scoring_run.download_file(output_prediction_file, output_file_path=output_prediction_file)
    predicted_labels_with_filename = {}
    with open(output_prediction_file) as predictions:
        predict_str = predictions.readline()
        target_class_mit_indoor = json.loads(predict_str)["labels"]
        while predict_str: 
            prediction_json_line = json.loads(predict_str)
            labels = prediction_json_line["labels"]
            probs = prediction_json_line["probs"]
            
            max_prob = max(probs)
            max_index = probs.index(max_prob)
            label_with_max_prob = labels[max_index]
            
            predicted_labels_with_filename[prediction_json_line["filename"]] = label_with_max_prob
            predict_str = predictions.readline()
    return predicted_labels_with_filename

# ground truth
def get_ground_truth(experiment_name: str) -> dict:
    default_ds = ws.datastores['workspaceblobstore']
    image_class_list_file_path = 'fhl/datasets/'+ test['ExperimentName'] + '/label/labeleddatapoints_test.jsonl'
    # creating groudtruth dictionary
    default_ds.download('.', prefix=image_class_list_file_path, overwrite=True)
    with open(image_class_list_file_path) as fp:
        number_of_lines_in_validation_dataset = len(fp.readlines())
        print("Number of lines in validation dataset: {}".format(number_of_lines_in_validation_dataset))
    ground_truth_with_filename = {}
    with open(image_class_list_file_path) as ground_truth:
        ground_truth_str = ground_truth.readline()
        while ground_truth_str:
            ground_truth_json_line = json.loads(ground_truth_str)
            ground_truth_with_filename[ground_truth_json_line["image_url"].replace("AmlDatastore://", "")] = ground_truth_json_line["label"]
            ground_truth_str = ground_truth.readline()
    return ground_truth_with_filename

In [ ]:
for test in classification_tests:
    prediction_dict = get_predictions(test['ExperimentName'], test['SoreRunId'])
    ground_truth_dict = get_ground_truth(test['ExperimentName'])
    assert len(ground_truth_dict) == len(prediction_dict)
    for filename in prediction_dict.keys():
        if filename in ground_truth_dict.keys():
            test['y_pred'].append(prediction_dict[filename])
            test['y_true'].append(ground_truth_dict[filename])
        else:
            print('Missing {} in ground trouth'.format(filename))

In [ ]:
for test in classification_tests:
    print('{} {}'.format(len(test['y_pred']), len(test['y_true'])))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

col = len(classification_tests)
fig, ax = plt.subplots(1, col)
fig.set_figwidth(8 * col)
# fig.set_figheight(FIG_HEIGHT * total_rows)
ax_index = 0
for test in classification_tests:
    cm = confusion_matrix(test['y_true'], test['y_pred'], labels= ['Lung Opacity', 'No Lung Opacity / Not Normal', 'Normal'])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Lung Opacity', 'No Lung Opacity / Not Normal', 'Normal'])
    disp.plot(values_format='d', ax=ax[ax_index])
    ax_index += 1


In [ ]:
for test in classification_tests:
    print(test['ExperimentName'])
    print(classification_report(test['y_true'], test['y_pred'], target_names =  ['Lung Opacity', 'No Lung Opacity / Not Normal', 'Normal']))
    print("=======================================================")
